# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [293]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### 데이터 읽어오기


In [294]:
RANDOM_STATE = 110

train_data = pd.read_csv("../../data/train_data.csv")
test_data = pd.read_csv("../../data/test_data.csv")

---

반복적으로 쓰는 툴 함수화

In [295]:
def plot_box(df, col_name):
    """
    주어진 데이터프레임과 열 이름에 대해 박스 플롯을 그리는 함수.

    Parameters:
    dataframe (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    """
    plt.figure(figsize=(7, 4))
    plt.boxplot(df[col_name], vert=False)
    plt.xlabel(col_name)
    plt.title(f'Box Plot of {col_name}')
    plt.show()

In [296]:
def value_counts_ratio(df, col_name, target_name='target'):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 비율과 갯수, 총 갯수를 출력하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    """
    # 각 값마다 타겟 변수의 비율 계산
    value_counts = df.groupby(col_name)[target_name].value_counts(normalize=True).unstack().fillna(0)
    
    # 각 값마다 타겟 변수의 갯수 계산
    counts = df.groupby(col_name)[target_name].value_counts().unstack().fillna(0)
    
    # 각 값마다 총 갯수 계산
    total_counts = df[col_name].value_counts().rename('Total_Count')
    
    # 비율과 갯수를 합침
    result = value_counts.join(counts, lsuffix='_ratio', rsuffix='_count')
    
    # 총 갯수를 합침
    result = result.join(total_counts, on=col_name)
    
    # 출력 형식 조정
    result.index.name = 'variable'
    print(f"\n{col_name}별 {target_name} 비율 및 갯수\n")
    print(result.rename(columns=lambda x: x.split('_')[0]))

In [297]:
def summarize_group(df, group_by_columns):
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_columns)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # 'target' 변수의 'AdNormal' 비율과 갯수 계산
        adnormal_count = group['target'].value_counts().get('AbNormal', 0)
        adnormal_ratio = adnormal_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, adnormal_count, adnormal_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', "'AdNormal' count", 'ratio', 'Total'])
    
    # 그룹화된 변수들의 이름을 제목행으로 출력
    print(f"Grouped by: {', '.join(group_by_columns)}")
    print()
    # 데이터프레임 출력
    print(results_df)

# 예시코드
# summarize_grouped_data(train_data, ['1st Pressure Collect Result_AutoClave', '1st Pressure Unit Time_AutoClave'])

In [298]:
def plot_ratio(df, group_by_column, target_column='target', abnormal_value='AbNormal'):
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_column)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # 'target' 변수의 'AbNormal' 비율과 갯수 계산
        abnormal_count = group[target_column].value_counts().get(abnormal_value, 0)
        abnormal_ratio = abnormal_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, abnormal_count, abnormal_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', f"'{abnormal_value}' count", 'ratio', 'Total'])
    
    # 그래프 크기 설정
    plt.figure(figsize=(10, 5))
    
    # 막대 그래프 생성
    ax = results_df.plot(kind='bar', x='group', y='ratio', legend=False)
    
    # 각 막대 위에 AbNormal 갯수와 총 갯수 표시
    for i, (abnormal_count, total) in enumerate(zip(results_df[f"'{abnormal_value}' count"], results_df['Total'])):
        ax.text(i, results_df['ratio'][i], f'{abnormal_count} ({total})', ha='center', va='bottom', fontsize=8)
    
     # 그래프 제목 및 축 레이블 설정
    ax.set_title(f'{abnormal_value} Ratio by {group_by_column}')
    ax.set_xlabel(group_by_column)
    ax.set_ylabel(f'{abnormal_value} Ratio')
   
    # 그래프 출력
    plt.show()

In [299]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_ratio_boxplot(data, time_ratio_column, target_column='target'):
    # 그래프 스타일 설정
    sns.set(style="whitegrid")

    # 그래프 그리기
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=time_ratio_column, y=target_column, data=data)

    # 그래프 제목 및 레이블 설정
    plt.title(f'{time_ratio_column} vs {target_column}')
    plt.xlabel(time_ratio_column)
    plt.ylabel(target_column)

    # 그래프 출력
    plt.show()

# 함수 호출 예제
#plot_time_ratio_vs_target(train_data, 'time_ratio_Dam')

---

### 1. Workorder

xxxxx-0001 -> xxxxx-1 변환 코드

In [300]:
# 'Workorder_AutoClave' 열에서 '-' 다음 숫자 값 추출 및 '000' 제거
train_data['Workorder'] = train_data['Workorder'].str.replace(r'-(\d+)', lambda x: '-' + x.group(1).lstrip('0'), regex=True)
test_data['Workorder'] = test_data['Workorder'].str.replace(r'-(\d+)', lambda x: '-' + x.group(1).lstrip('0'), regex=True)

아래의 Workorder에 대한 비율에 따른 범주화 관련 내용은 회의때 다뤄봐야 될듯

In [ ]:
# # 타겟 변수를 0과 1로 변환
# train_data['target_binary'] = train_data['target'].apply(lambda x: 1 if x == 'AbNormal' else 0)

# # Workorder 변수의 값에 대한 타겟 변수 비율 계산
# workorder_target_ratio = train_data.groupby('Workorder')['target_binary'].mean()

# # 불필요한 변수 제거
# train_data.drop(['target_binary'], axis=1, inplace=True)

# # 파생 변수 생성 함수
# def create_derived_variable(row, ratio_dict, threshold):
#     return 1 if ratio_dict.get(row['Workorder'], 0) >= threshold else 0

# # 파생 변수 생성
# train_data['Workorder_0.9'] = train_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.9)
# train_data['Workorder_0.7'] = train_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.7)
# train_data['Workorder_0.5'] = train_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.5)

# test_data['Workorder_0.9'] = test_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.9)
# test_data['Workorder_0.7'] = test_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.7)
# test_data['Workorder_0.5'] = test_data.apply(create_derived_variable, axis=1, ratio_dict=workorder_target_ratio, threshold=0.5)

In [ ]:
# # 결과 확인
# value_counts_ratio(train_data, 'Workorder_0.9')
# value_counts_ratio(train_data, 'Workorder_0.7')
# value_counts_ratio(train_data, 'Workorder_0.5')

### 2. Dispenser_num

원본 데이터상으로는 Equiment 변수  
Dispneser_num 값으로 0 #1 #2 값 -> 원핫인코딩으로 변경

Dispenser 값을 범주로서 인식이 아닌 #1 #2 에 따라 학습되는 행이 달라지게 하기 위함

In [304]:
# Dispenser_num 값에 따라 새로운 변수 생성
train_data['Dispenser_1'] = train_data['Dispenser_num'].apply(lambda x: 1 if x == '#1' else 0)
train_data['Dispenser_2'] = train_data['Dispenser_num'].apply(lambda x: 1 if x == '#2' else 0)

test_data['Dispenser_1'] = test_data['Dispenser_num'].apply(lambda x: 1 if x == '#1' else 0)
test_data['Dispenser_2'] = test_data['Dispenser_num'].apply(lambda x: 1 if x == '#2' else 0)

# 불필요한 변수 제거
train_data.drop(['Dispenser_num'], axis=1, inplace=True)
test_data.drop(['Dispenser_num'], axis=1, inplace=True)


['Dispenser_num', 'Dispenser_1', 'Dispenser_2']별 target 비율 및 갯수

                                       AbNormal    Normal  AbNormal   Normal  \
Dispenser_num Dispenser_1 Dispenser_2                                          
#0            0           0            1.000000  0.000000      34.0      0.0   
#1            1           0            0.058614  0.941386    1466.0  23545.0   
#2            0           1            0.054977  0.945023     850.0  14611.0   

                                       Total  
Dispenser_num Dispenser_1 Dispenser_2         
#0            0           0               34  
#1            1           0            25011  
#2            0           1            15461  


### 3. WorkMode Collect Result

In [305]:
# 그룹화할 변수들
groupby_columns = [
    "WorkMode Collect Result_Dam"
    ,"WorkMode Collect Result_Fill1"
    , "WorkMode Collect Result_Fill2"
    ]

# 그룹화하여 target 변수 값의 비율을 계산
grouped = train_data.groupby(groupby_columns)["target"].value_counts(normalize=True).unstack().fillna(0)

# 결과 출력
print(grouped)

target                                                                                   AbNormal  \
WorkMode Collect Result_Dam WorkMode Collect Result_Fill1 WorkMode Collect Result_Fill2             
7.0                         7.0                           0.0                            0.073326   

target                                                                                     Normal  
WorkMode Collect Result_Dam WorkMode Collect Result_Fill1 WorkMode Collect Result_Fill2            
7.0                         7.0                           0.0                            0.926674  


하나의 그룹으로 묶이고 이때 하나의 고윳값만을 가짐  
-> 한개의 변수만 냅두고 나머지 drop, 냅둔 변수에 대해서는 값이 있음면 1 없으면 0을 부여

In [306]:
# WorkMode Collect Result_Dam의 이름을 WorkMode Collect Result로 변경
train_data = train_data.rename(columns={'WorkMode Collect Result_Dam': 'WorkMode Collect Result'})
test_data = test_data.rename(columns={'WorkMode Collect Result_Dam': 'WorkMode Collect Result'})

# WorkMode Collect Result_Fill1, WorkMode Collect Result_Fill2 열 드롭
train_data = train_data.drop(columns=['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'])
test_data = test_data.drop(columns=['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'])

In [307]:
# WorkMode Collect Result 열의 값이 7인 행을 1로 변경
train_data['WorkMode Collect Result'] = train_data['WorkMode Collect Result'].replace(7, 1)
test_data['WorkMode Collect Result'] = test_data['WorkMode Collect Result'].replace(7, 1)

# WorkMode Collect Result 열의 결측값을 0으로 채움
train_data['WorkMode Collect Result'] = train_data['WorkMode Collect Result'].fillna(0)
test_data['WorkMode Collect Result'] = test_data['WorkMode Collect Result'].fillna(0)

In [308]:
test_data['WorkMode Collect Result'].value_counts()

0.0    24059
1.0    16447
Name: WorkMode Collect Result, dtype: int64

### 4. Receip No Collect Result

In [309]:
summarize_group(train_data, [
    'Receip No Collect Result_Dam'
    , 'Receip No Collect Result_Fill1'
    , 'Receip No Collect Result_Fill2'])

Grouped by: Receip No Collect Result_Dam, Receip No Collect Result_Fill1, Receip No Collect Result_Fill2

          group  'AdNormal' count     ratio  Total
0     (1, 1, 1)              2215  0.056396  39276
1     (1, 4, 1)                 1  1.000000      1
2     (1, 6, 6)                 3  1.000000      3
3     (3, 3, 3)                43  0.671875     64
4     (6, 6, 6)                82  0.083673    980
5     (9, 9, 9)                 1  0.010417     96
6  (17, 17, 17)                 5  0.058140     86


다른값을 가지면 AbNormal ratio 1 임을 위의 결과로 확인 가능

In [310]:
# 세 변수의 값이 동일하면 해당 값을 가져가고, 하나라도 일치하지 않으면 0의 값을 가지는 파생 변수 생성 함수
def create_receip_no_collect_result(df):
    df['Receip_No_Collect_Result'] = df.apply(
        lambda row: row['Receip No Collect Result_Dam'] 
                    if (row['Receip No Collect Result_Dam'] == row['Receip No Collect Result_Fill1'] == row['Receip No Collect Result_Fill2']) 
                    else 0, 
        axis=1
    )

# 함수 적용
create_receip_no_collect_result(train_data)
create_receip_no_collect_result(test_data)

In [311]:
value_counts_ratio(train_data, 'Receip_No_Collect_Result')


Receip_No_Collect_Result별 target 비율 및 갯수

          AbNormal    Normal  AbNormal   Normal  Total
variable                                              
0         1.000000  0.000000       4.0      0.0      4
1         0.056396  0.943604    2215.0  37061.0  39276
3         0.671875  0.328125      43.0     21.0     64
6         0.083673  0.916327      82.0    898.0    980
9         0.010417  0.989583       1.0     95.0     96
17        0.058140  0.941860       5.0     81.0     86


In [312]:
# 제거할 변수 목록
columns_to_drop = [
    'Receip No Collect Result_Dam',
    'Receip No Collect Result_Fill1',
    'Receip No Collect Result_Fill2'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 5. PalletID_Collect_Result

In [313]:
# 결과 확인
summarize_group(train_data,[
    'PalletID Collect Result_Dam'
    , 'PalletID Collect Result_Fill1'
    , 'PalletID Collect Result_Fill2'])

Grouped by: PalletID Collect Result_Dam, PalletID Collect Result_Fill1, PalletID Collect Result_Fill2

           group  'AdNormal' count     ratio  Total
0      (1, 1, 1)               162  0.051494   3146
1      (1, 1, 4)                 1  1.000000      1
2      (1, 1, 7)                 1  1.000000      1
3      (1, 1, 9)                 1  1.000000      1
4      (1, 2, 2)                 1  1.000000      1
..           ...               ...       ...    ...
87  (16, 10, 10)                 1  1.000000      1
88  (16, 15, 15)                 1  1.000000      1
89   (16, 16, 8)                 1  1.000000      1
90  (16, 16, 11)                 1  1.000000      1
91  (16, 16, 16)               110  0.062007   1774

[92 rows x 4 columns]


PalletID 도 마찬가지로 3개의 값이 일치하지 않을 경우 AbNormal ratio 값이 1로 확인됨

In [314]:
# 세 변수의 값이 동일하면 해당 값을 가져가고, 하나라도 일치하지 않으면 0의 값을 가지는 파생 변수 생성 함수
def create_palletid_collect_result(df):
    df['PalletID_Collect_Result'] = df.apply(
        lambda row: row['PalletID Collect Result_Dam'] 
                    if (row['PalletID Collect Result_Dam'] == row['PalletID Collect Result_Fill1'] == row['PalletID Collect Result_Fill2']) 
                    else 0, 
        axis=1
    )

# 함수 적용
create_palletid_collect_result(train_data)
create_palletid_collect_result(test_data)

In [315]:
# 결과 확인
value_counts_ratio(train_data,'PalletID_Collect_Result')


PalletID_Collect_Result별 target 비율 및 갯수

          AbNormal    Normal  AbNormal  Normal  Total
variable                                             
0         1.000000  0.000000      83.0     0.0     83
1         0.051494  0.948506     162.0  2984.0   3146
2         0.066859  0.933141     209.0  2917.0   3126
3         0.063171  0.936829     200.0  2966.0   3166
4         0.053875  0.946125     171.0  3003.0   3174
5         0.054115  0.945885     167.0  2919.0   3086
6         0.058125  0.941875     181.0  2933.0   3114
7         0.052922  0.947078     163.0  2917.0   3080
8         0.057412  0.942588     177.0  2906.0   3083
9         0.045929  0.954071      88.0  1828.0   1916
10        0.051117  0.948883     103.0  1912.0   2015
11        0.057158  0.942842     109.0  1798.0   1907
12        0.042394  0.957606      85.0  1920.0   2005
13        0.068766  0.931234     131.0  1774.0   1905
14        0.048485  0.951515      96.0  1884.0   1980
15        0.059096  0.940904     115.0  

In [316]:
# 제거할 변수 목록
columns_to_drop = [
    'PalletID Collect Result_Dam',
    'PalletID Collect Result_Fill1',
    'PalletID Collect Result_Fill2'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 6. Production Qty Collect Result

In [317]:
# 결과 확인
summarize_group(train_data,[
    'Production Qty Collect Result_Dam'
    , 'Production Qty Collect Result_Fill1'
    , 'Production Qty Collect Result_Fill2'])

Grouped by: Production Qty Collect Result_Dam, Production Qty Collect Result_Fill1, Production Qty Collect Result_Fill2

               group  'AdNormal' count     ratio  Total
0          (0, 0, 0)                13  0.141304     92
1         (0, 0, 12)                 1  1.000000      1
2        (0, 0, 300)                 1  1.000000      1
3          (0, 2, 2)                 1  1.000000      1
4        (0, 19, 19)                 1  1.000000      1
..               ...               ...       ...    ...
695  (604, 604, 604)                 0  0.000000      1
696  (605, 605, 605)                 0  0.000000      1
697  (606, 606, 606)                 0  0.000000      1
698  (607, 607, 607)                 0  0.000000      1
699  (608, 608, 608)                 0  0.000000      1

[700 rows x 4 columns]


In [318]:
# 세 변수의 값이 동일하면 해당 값을 가져가고, 하나라도 일치하지 않으면 0의 값을 가지는 파생 변수 생성 함수
def create_palletid_collect_result(df):
    df['Production_Qty_Collect_Result'] = df.apply(
        lambda row: row['Production Qty Collect Result_Dam'] 
                    if (row['Production Qty Collect Result_Dam'] == row['Production Qty Collect Result_Fill1'] == row['Production Qty Collect Result_Fill2']) 
                    else 0, 
        axis=1
    )

# 함수 적용
create_palletid_collect_result(train_data)
create_palletid_collect_result(test_data)

In [319]:
# 결과 확인
value_counts_ratio(train_data,'Production_Qty_Collect_Result')


Production_Qty_Collect_Result별 target 비율 및 갯수

          AbNormal    Normal  AbNormal  Normal  Total
variable                                             
0         0.572973  0.427027     106.0    79.0    185
1         0.132075  0.867925      14.0    92.0    106
2         0.059829  0.940171       7.0   110.0    117
3         0.076271  0.923729       9.0   109.0    118
4         0.034783  0.965217       4.0   111.0    115
...            ...       ...       ...     ...    ...
604       0.000000  1.000000       0.0     1.0      1
605       0.000000  1.000000       0.0     1.0      1
606       0.000000  1.000000       0.0     1.0      1
607       0.000000  1.000000       0.0     1.0      1
608       0.000000  1.000000       0.0     1.0      1

[607 rows x 5 columns]


0값이 공정별 Production_Qty_Collect_Result 값이 같지 않는 경우에 해당  
무조건으로 AbNormal 비율이 1은 아니지만 0.57 값을 가지며 상대적으로 높음으로 0값을 취하도록 함

In [320]:
# 제거할 변수 목록
columns_to_drop = [
    'Production Qty Collect Result_Dam',
    'Production Qty Collect Result_Fill1',
    'Production Qty Collect Result_Fill2'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 7. Judge Value

In [321]:
# "Judge Value"를 포함하는 변수 필터링
judge_value_columns = [col for col in train_data.columns if "Judge Value" in col]

# 필터링된 변수 출력
print(judge_value_columns)

['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam', 'Chamber Temp. Judge Value_AutoClave', 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2']


In [322]:
value_counts_ratio(train_data, 'Chamber Temp. Judge Value_AutoClave')


Chamber Temp. Judge Value_AutoClave별 target 비율 및 갯수

          AbNormal    Normal  AbNormal  Normal  Total
variable                                             
NG        0.057135  0.942865       651   10743  11394
OK        0.058361  0.941639      1699   27413  29112


In [323]:
# "Chamber Temp. Judge Value_AutoClave" 변수의 값을 기준으로 파생 변수 생성 함수
def create_judge_value_binary(df):
    df['Chamber_Temp_OKNG'] = df['Chamber Temp. Judge Value_AutoClave'].apply(
        lambda x: 1 if x == 'OK' else 0
    )

# 함수 적용
create_judge_value_binary(train_data)
create_judge_value_binary(test_data)

In [324]:
value_counts_ratio(train_data, 'Chamber_Temp_OKNG')


Chamber_Temp_OKNG별 target 비율 및 갯수

          AbNormal    Normal  AbNormal  Normal  Total
variable                                             
0         0.057135  0.942865       651   10743  11394
1         0.058361  0.941639      1699   27413  29112


'Chamber Temp. Judge Value_AutoClave'(Chamber_Temp_OKNG) 는 NG 값과 OK 값을 가짐  
-> 'Chamber Temp. Judge Value_AutoClave' 변수를 제외한 나머지 변수 확인

In [325]:
summarize_group(train_data, [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam'
    , 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2'])

Grouped by: HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam, GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave, HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1, HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2

              group  'AdNormal' count     ratio  Total
0  (OK, OK, OK, OK)               496  0.043921  11293


OK 값을 고유값으로 가짐 -> OK 값을 가지면 1 아니면 0을 부여

In [326]:
# 4개의 변수 목록
judge_value_columns = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2'
]

# 파생 변수 생성 함수
def create_judge_value_feature(df):
    df['Judge_Value_OK'] = df[judge_value_columns].apply(
        lambda row: 1 if any(row == 'OK') else 0, 
        axis=1
    )

# 함수 적용
create_judge_value_feature(train_data)
create_judge_value_feature(test_data)

In [327]:
value_counts_ratio(train_data, 'Judge_Value_OK')


Judge_Value_OK별 target 비율 및 갯수

          AbNormal    Normal  AbNormal  Normal  Total
variable                                             
0         0.063465  0.936535      1854   27359  29213
1         0.043921  0.956079       496   10797  11293


In [328]:
summarize_group(train_data, [
    'Chamber_Temp_OKNG'
    ,'Judge_Value_OK'])

Grouped by: Chamber_Temp_OKNG, Judge_Value_OK

    group  'AdNormal' count     ratio  Total
0  (0, 0)               512  0.062645   8173
1  (0, 1)               139  0.043154   3221
2  (1, 0)              1342  0.063783  21040
3  (1, 1)               357  0.044227   8072


ok 관련 값을 가지는 변수는 위의 2가지 경우  
-> 전처리 과정에서 불필요한 변수라면 drop해도 괜찮을듯

In [330]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 114 entries, Model.Suffix to Judge_Value_OK
dtypes: float64(56), int64(49), object(9)
memory usage: 35.2+ MB


In [331]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 114 entries, Model.Suffix to Judge_Value_OK
dtypes: float64(56), int64(49), object(9)
memory usage: 35.2+ MB
